# Import libraries

In [ ]:
#Importing the Libraries
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


#Load data 
Tên data: customer-segmentation.csv \
Gồm : 2240 hàng × 29 cột



**People**

ID: Customer's unique identifier \
Year_Birth: Customer's birth year \
Education: Customer's education level \
Marital_Status: Customer's marital status \
Income: Customer's yearly household income \
Kidhome: Number of children in customer's household \
Teenhome: Number of teenagers in customer's household \
Dt_Customer: Date of customer's enrollment with the company \
Recency: Number of days since customer's last purchase \
Complain: 1 if customer complained in the last 2 years, 0 otherwise \

**Products**

MntWines: Amount spent on wine in last 2 years \
MntFruits: Amount spent on fruits in last 2 years \
MntMeatProducts: Amount spent on meat in last 2 years \
MntFishProducts: Amount spent on fish in last 2 years \
MntSweetProducts: Amount spent on sweets in last 2 years \
MntGoldProds: Amount spent on gold in last 2 years \

**Promotion**

NumDealsPurchases: Number of purchases made with a discount \
AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise \
AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise \
AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise \
AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise \
AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise \
Response: 1 if customer accepted the offer in the last campaign, 0 otherwise \

**Place**

NumWebPurchases: Number of purchases made through the company’s web site \
NumCatalogPurchases: Number of purchases made using a catalogue \
NumStorePurchases: Number of purchases made directly in stores \
NumWebVisitsMonth: Number of visits to company’s web site in the last month

In [ ]:
data= pd.read_csv("customer-segmentation.csv",sep="\t")
data.head()

In [ ]:
data.shape

#Data Cleaning

In [ ]:
#Information on features 
data.info()

Từ kết quả trên, chúng ta có thể kết luận và lưu ý rằng:

Có những giá trị bị thiếu trong thu nhập \
Dt_Customer cho biết ngày khách hàng tham gia nhưng không được phân tích cú pháp thành DateTime \
Có một số feature phân loại trong khung dữ liệu; vì có một số tính năng trong dtype: object) \
=>  Vì vậy, chúng ta sẽ cần phải mã hóa chúng thành dạng số.


**Chúng ta tiến hành loại bỏ các hàng có dữ liệu bị thiếu**

In [ ]:
#To remove the NA values
data = data.dropna()
print("The total number of data-points after removing the rows with missing values are:", len(data))

**Đưa feature Dt_customer về dạng DateTime**

In [ ]:
data["Dt_Customer"] = pd.to_datetime(data["Dt_Customer"])
dates = []
for i in data["Dt_Customer"]:
    i = i.date()
    dates.append(i)  
#Dates of the newest and oldest recorded customer
print("The newest customer's enrolment date in therecords:",max(dates))
print("The oldest customer's enrolment date in the records:",min(dates))

**Tạo một featue ("Customer_For") về số ngày khách hàng bắt đầu mua sắm trong cửa hàng so với ngày được ghi cuối cùng**

In [ ]:
#Created a feature "Customer_For"
days = []
d1 = max(dates) #taking it to be the newest customer
for i in dates:
    delta = d1 - i
    days.append(delta)
data["Customer_For"] = days
data["Customer_For"] = pd.to_numeric(data["Customer_For"], errors="coerce")

**Sau đây chúng ta sẽ khám phá các Uniqe value trong các feature để có được ý tưởng rõ ràng về dữ liệu.**

In [ ]:
print("Total categories in the feature Marital_Status:\n", data["Marital_Status"].value_counts(), "\n")
print("Total categories in the feature Education:\n", data["Education"].value_counts())

Trong phần tiếp theo, tôi sẽ thực hiện các bước sau để thiết kế một số tính năng mới:

Trích xuất "Age" của khách hàng theo "Year_Birth" cho biết năm sinh của người tương ứng. \
Tạo một feature "Spent" cho biết tổng số tiền khách hàng đã chi tiêu trong các danh mục khác nhau trong khoảng thời gian hai năm. \
Tạo một tính năng khác "Living_With" ngoài "Marital_Status" để bóc tách hoàn cảnh sống của các cặp vợ chồng. \
Tạo feature "Children" để biểu thị tổng số trẻ em trong một hộ gia đình, đó là trẻ em và thanh thiếu niên. \
Để hiểu rõ hơn về hộ gia đình, hãy tạo tính năng cho biết "Family_Size" \
Tạo một tính năng "Is_Parent" để biểu thị trạng thái làm cha mẹ \
Cuối cùng, tôi sẽ tạo ba danh mục trong "Education_" bằng cách đơn giản hóa số lượng giá trị của nó. \
Bỏ một số tính năng thừa

In [ ]:
#Feature Engineering
#Age of customer today 
data["Age"] = 2021-data["Year_Birth"]

#Total spendings on various items
data["Spent"] = data["MntWines"]+ data["MntFruits"]+ data["MntMeatProducts"]+ data["MntFishProducts"]+ data["MntSweetProducts"]+ data["MntGoldProds"]

#Deriving living situation by marital status"Alone"
data["Living_With"]=data["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})

#Feature indicating total children living in the household
data["Children"]=data["Kidhome"]+data["Teenhome"]

#Feature for total members in the householde
data["Family_Size"] = data["Living_With"].replace({"Alone": 1, "Partner":2})+ data["Children"]

#Feature pertaining parenthood
data["Is_Parent"] = np.where(data.Children> 0, 1, 0)

#Segmenting education levels in three groups
data["Education"]=data["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

#For clarity
data=data.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})

#Dropping some of the redundant features
to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"]
data = data.drop(to_drop, axis=1)

In [ ]:
data.describe()

Các số liệu thống kê trên cho thấy một số khác biệt về Thu nhập trung bình và Tuổi và Thu nhập tối đa và độ tuổi.

Có một lưu ý rằng độ tuổi tối đa là 128 tuổi, Như  đã tính toán độ tuổi sẽ là ngày hôm nay (tức là năm 2021) và dữ liệu đã cũ.

Vì vậy chúng ta phải xem xét tầm nhìn rộng hơn của dữ liệu.

**Plot data**

In [ ]:
#To plot some selected features 
#Setting up colors prefrences
sns.set(rc={"axes.facecolor":"#FFF9ED","figure.facecolor":"#FFF9ED"})
pallet = ["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"]
cmap = colors.ListedColormap(["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"])
#Plotting following features
To_Plot = [ "Income", "Recency", "Customer_For", "Age", "Spent", "Is_Parent"]
print("Reletive Plot Of Some Selected Features: A Data Subset")
plt.figure()
sns.pairplot(data[To_Plot], hue= "Is_Parent",palette= (["#682F2F","#F3AB60"]))
#Taking hue 
plt.show()

Có một số outlier trong các feature Income và Age. \
=> Tôi sẽ xóa các outlier trong dữ liệu.

In [ ]:
#Dropping the outliers by setting a cap on Age and income. 
data = data[(data["Age"]<90)]
data = data[(data["Income"]<600000)]
print("The total number of data-points after removing the outliers are:", len(data))

**Tiếp theo, chúng ta hãy xem xét correlation giữa các features.**

In [ ]:
#correlation matrix
corrmat= data.corr()
mask = np.triu(np.ones_like(corrmat, dtype=bool))

# Create a custom divergin palette

plt.figure(figsize=(20,20))  
sns.heatmap(corrmat,annot=True,mask=mask, cmap=cmap,fmt='.2f', center=0, square=True)

# Data visualisation using Autoviz

AUTOVIZ install

In [ ]:
!pip install autoviz

In [ ]:
! pip install xlrd

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()
df_av = AV.AutoViz('/content/customer-segmentation.csv',sep="\t")

# Data visualisation

In [ ]:
data['Education'].value_counts()

In [ ]:
sns.countplot(x = 'Education' , data = data)
plt.show()

=> Chưa tốt nghiệp chiếm số lượng thấp nhất


In [ ]:
data['Living_With'].value_counts()

In [ ]:
sns.countplot( x = 'Living_With',data = data)
plt.show()

=> Phần đông là người đã lập gia đình

In [ ]:
obj = ['Education','Living_With']
for i in obj:
    data=data.copy()
    data.groupby(i)['Age'].mean().plot.bar()
    plt.xlabel(i)
    plt.ylabel('Age')
    plt.title(i)
    plt.show()

In [ ]:
# I found no relations
for i in range(len(obj)):
    for j in range(2):
        x = obj[i]
        if obj[j] != x:
            sns.barplot(x= x,y='Age',hue=obj[j],data=data)
            plt.legend(bbox_to_anchor=(1.1, 1.05))
            plt.show()

In [ ]:
df7 = data.copy()

In [ ]:
for i in range(len(obj)):
    x='Living_With'
    for j in range(1):
        if obj[i] != x:
            sns.barplot(x= x,y='Age',hue=obj[i],data=df7)
            sns.set(rc={'figure.figsize':(11,12)})
            plt.show()

In [ ]:
for i in range(len(obj)):
    x='Living_With'
    for j in range(1):
        if obj[i] != x:
            sns.barplot(x= x,y='Income',hue=obj[i],data=df7)
            sns.set(rc={'figure.figsize':(11,12)})
            plt.show()

In [ ]:
y = data.drop(['Education','Living_With'],axis = 1)
for i in y.columns:
    sns.boxplot(x = i, data = y,color = 'yellowgreen')   
    plt.xlabel(i)
    plt.show()

# Data Preprocessing

Label encoding the categorical features\
Scaling the features using the standard scaler\
Creating a subset dataframe for  dimensionality reduction

Đếm số outliers dùng phương pháp IQR method

In [ ]:
def count_outliers(data,col):
        q1 = data[col].quantile(0.25,interpolation='nearest')
        q2 = data[col].quantile(0.5,interpolation='nearest')
        q3 = data[col].quantile(0.75,interpolation='nearest')
        q4 = data[col].quantile(1,interpolation='nearest')
        IQR = q3 -q1
        global LLP
        global ULP
        LLP = q1 - 1.5*IQR
        ULP = q3 + 1.5*IQR
        if data[col].min() > LLP and data[col].max() < ULP:
            print("No outliers in",i)
        else:
            print("There are outliers in",i)
            x = data[data[col]<LLP][col].size
            y = data[data[col]>ULP][col].size
            a.append(i)
            print('Count of outliers are:',x+y)
global a
a = []
for i in y.columns:
    count_outliers(df,i)

In [ ]:
#Get list of categorical variables 
s = (data.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables in the dataset:", object_cols)


Gán mỗi giá trị của đăng trưng dạng nhóm thành một số

In [ ]:
#Label Encoding the object dtypes.
LE=LabelEncoder()
for i in object_cols:
    data[i]=data[[i]].apply(LE.fit_transform)
    
print("All features are now numerical")

Chuẩn hóa các đặc trưng của dữ liệu

In [ ]:
#Creating a copy of data
ds = data.copy()
# creating a subset of dataframe by dropping the features on deals accepted and promotions
cols_del = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1','AcceptedCmp2', 'Complain', 'Response']
ds = ds.drop(cols_del, axis=1)
#Scaling
scaler = StandardScaler()
scaler.fit(ds)
scaled_ds = pd.DataFrame(scaler.transform(ds),columns= ds.columns )
print("All features are now scaled")

In [ ]:
#Scaled data to be used for reducing the dimensionality
print("Dataframe to be used for further modelling:")
scaled_ds.head()

Dimentionality Reduction

Trong bài toán này có nhiều yếu tố trên cơ sở đó sẽ thực hiện phân loại cuối cùng. Các yếu tố này về cơ bản là thuộc tính hoặc tính năng. Số lượng tính năng càng cao thì càng khó làm việc với nó. Nhiều tính năng trong số này có tương quan và do đó dư thừa. Đây là lý do tại sao tôi sẽ thực hiện giảm kích thước trên các tính năng đã chọn trước khi đưa chúng qua bộ phân loại.\
Giảm thứ nguyên là quá trình giảm số lượng các biến ngẫu nhiên đang được xem xét, bằng cách thu được một tập hợp các biến chính.

Phân tích thành phần chính (PCA) là một kỹ thuật để giảm kích thước của các tập dữ liệu đó, tăng khả năng diễn giải nhưng đồng thời giảm thiểu mất mát thông tin.

Các bước trong phần này:

Giảm kích thước với PCA\
Vẽ khung dữ liệu đã giảm\
Giảm kích thước với PCA

Ở đây giảm kích thước xuống còn 3.

In [ ]:
df1 = df.groupby('Age').agg({ 'Education' : 'count', 'Marital_Status' : 'count', 'Income':'mean', 'Kidhome' : 'count',
       'Teenhome' : 'count',  'Recency':'mean', 
        'NumDealsPurchases' : 'mean', 'NumWebPurchases' : 'mean',
       'NumCatalogPurchases' : 'mean', 'NumStorePurchases' : 'mean', 'NumWebVisitsMonth' : 'mean',
        'Complain' : 'sum', 'Response' : 'sum', 'Spent' : 'mean'})
df1


In [ ]:
# gropubu analysis using education
df2 = df.groupby('Education').agg({ 'Age' : 'mean', 'Education' : 'count', 'Marital_Status' : 'sum', 'Income':'mean', 'Kidhome' : 'count',
       'Teenhome' : 'count',  'Recency':'mean', 
        'NumDealsPurchases' : 'mean', 'NumWebPurchases' : 'mean',
       'NumCatalogPurchases' : 'mean', 'NumStorePurchases' : 'mean', 'NumWebVisitsMonth' : 'mean',
        'Complain' : 'sum', 'Response' : 'sum', 'Spent' : 'mean'})
df2
#2n Cycle       0
#Basic          1
#Graduation     2
#Master         3
#PhD            4




Đa số những người có bằng cử nhân có mức chi tiêu trung bình cao \
Những người bình thường thì có mức thu nhập thấp nên mức chi tiêu của họ cũng tương đối thấp trong nhóm\
Những người có bằng tiến sĩ chiếm thứ 2 trong nhóm nhưng có mức thu nhập và chi tiêu nhiều hơn hẳn so với còn lại\
Có thể thấy càng có thu nhập cao thì mức chi tiêu càng lớn.

In [ ]:
px.bar(data_frame=df2.drop(['Income','Marital_Status'],axis = 1), barmode='group',
       title = "<b>Education wise Analyzing</b>",template="plotly_dark")
# droping income,marital status  since they have lagrge value


Có thể thấy người có bằng tiến sĩ chi tiêu vượt trội hơn hẳn trong 2 năm gần đây.\
Nhưng người có bằng cử nhân là chi tiêu nhiều nhất do có số lượng nhiều hơn.

In [ ]:
px.bar(data_frame=df2[['Income','Marital_Status']], barmode='group',
       title = "<b>Education wise Analyzing</b>",template="plotly_dark")


In [ ]:
Thu nhập của người có bằng tiến sĩ nhiều hơn trong nhóm

In [ ]:
df3 = df.groupby('Marital_Status').agg({ 'Age' : 'mean', 'Education' : 'count', 'Marital_Status' : 'sum', 'Income':'mean', 'Kidhome' : 'count',
       'Teenhome' : 'count',  'Recency':'mean', 
        'NumDealsPurchases' : 'mean', 'NumWebPurchases' : 'mean',
       'NumCatalogPurchases' : 'mean', 'NumStorePurchases' : 'mean', 'NumWebVisitsMonth' : 'mean',
        'Complain' : 'sum', 'Response' : 'sum', 'Spent' : 'mean'})
df3

#Married       3
#Together      5
#Single        4
#Divorced      2
#Widow         6
#Alone         1
#YOLO          7
#Absurd        0

Đa số những mọi người trong nhóm là đã và đang lập gia đình 

In [ ]:
px.bar(data_frame=df3.drop(['Income','Marital_Status','Spent'],axis = 1), barmode='group',
       title = "<b>Marital status wise Analyzing</b>",template="plotly_dark")


Người có gia đình có trẻ đang đi học chiếm tỉ lệ lớn

In [ ]:
px.bar(data_frame=df3[['Income','Spent']], barmode='group',
       title = "<b>Marital_Status wise Analyzing</b>",template="plotly_dark")



Thu nhập trung bình của góa phụ thường nhiều và nhiều nhất là những người absurd\
Chi tiêu của những người absurd cũng nhiều nhất.
